# 연습 - 종목정보(코드,가격), 계좌잔고확인, 주문, 스케쥴링

## 주의사항
-------------------------------------
* 파이썬은 3.6 32비트 버전을 설치
* 프로젝트 경로 및 폴더명은 공백과 한글이 없도록 설치
* 사이보스는 관리자 권한으로 수행
* 주피터 노트북이 너무 느리면, VS Code에서 파이썬 플러그인을 설치해서 개발
* VS Code Python plugin에서 주피터 서버를 띄워줌. 수행속도는 차이가 없으나 코딩화면에서 반응 속도 빠름.
-------------------------------------

## 계좌잔고확인 및 주문

-------------------------------------

    ==========================================
    예수금                            :   1000
    매수율(월모멘텀+단기하락)^2/4*100 :    70%
    투자증거금 (예수금 * 매수율)      :    700
    계약당 증거금                     :    200
    투자증거금 / 계약당 증거금        :   3.50
    ==========================================
    매수 계약수                       :      3
    최대 계약수                       :     50
    max(매수,최대)                    :      3
    주문가능 수량                     :      5
    당기선물 잔존일이 0이면 매수하지 않는다.  

In [1]:
import sys
import win32com.client
import datetime
import sqlite3
import ctypes

g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
g_objCpTrade0721 = win32com.client.Dispatch('CpTrade.CpTd0721F')
g_objCpTrade0732 = win32com.client.Dispatch('CpTrade.CpTd0732')
g_objCpTrade6722 = win32com.client.Dispatch('CpTrade.CpTd6722')



# g_objFutureMgr = win32com.client.Dispatch("CpUtil.CpKFutureCode")
# g_objFutureMaster = win32com.client.Dispatch("Dscbo1.FutureMst")

def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # 주문 관련 초기화
    ret = g_objCpTrade.TradeInit(0)
    if (ret != 0):
        print("주문 초기화 실패, 오류번호 ", ret)
        return False


    # # 주문 관련 초기화
    # if (g_objCpTrade0721.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
         
 
    # # 주문 관련 초기화
    # if (g_objCpTrade0732.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
         
    return True
 

if __name__ == "__main__":
    if False == InitPlusCheck() :
        exit()


    print("==========================================")    
    accNum = g_objCpTrade.AccountNumber[0]  # 계좌번호
    print("계좌번호 : %s" % accNum)    

    g_objCpTrade0721.SetInputValue(0,accNum)
    g_objCpTrade0721.SetInputValue(1,"50")
    g_objCpTrade0721.SetInputValue(2,"106Q9")
    
    g_objCpTrade0721.SetInputValue(3,20)
    # 0 - (string)계좌번호
    # 1 - (string)상품관리구분[defaut:50]
    # 2 - (string) 종목코드[default:""]
    # 3 - (long) 요청건수[default:10] - 최대20개
    
    g_objCpTrade0721.BlockRequest()
 

    rqStatus = g_objCpTrade0721.GetDibStatus()
    rqRet = g_objCpTrade0721.GetDibMsg1()
    print("g_objCpTrade0721 통신상태", rqStatus, rqRet)
        


    # 3 - (longlong)현금결제예탁액
    # 4 - (longlong)대용결제예탁액
    # 5 - (longlong)총결제예탁액
    # 6 - (longlong)현금위탁증거금
    # 7 - (longlong)대용위탁증거금
    # 8 - (longlong)총위탁증거금
    # 9 - (longlong)현금주문가능액
    # 10 - (longlong)대용주문가능액
    # 11 - (longlong)총주문가능액
    # 12 - (longlong)현금인출가능액
    # 13 - (longlong)대용인출가능액
    # 14 - (longlong)총인출가능액
    # 15 - (longlong)순위험증거금
    # 17 - (longlong)체결분순익
    # 18 - (longlong)미체결주문분
    # 20 - (longlong)현금미결제예탁액
    # 21 - (longlong)대용미결제예탁액
    # 22 - (longlong)총미결제예탁액
    # 23 - (longlong)현금결재순손실
    # 24 - (longlong)총결재순손실
    # 25 - (long) 수신개수
    # 26- (long) 매도신규주문수량
    # 27- (long) 매도청산주문수량
    # 28- (long) 매수신규주문수량
    # 29- (long) 매수청산주문수량
    # 30- (longlong) 체결분순손실
    # 31 - (longlong) C등급총위험증거금
    # 32 - (longlong) C등급총위험가결산
    # 33 - (longlong) D등급총위험증거금
    # 34 - (longlong) D등급총위험가결산
    # 35 - (longlong) E등급총위험증거금
    # 36 - (longlong) E등급총위험가결산

    print("3 - (longlong)현금결제예탁액        : %d" % g_objCpTrade0721.GetHeaderValue(3 ))
    print("4 - (longlong)대용결제예탁액        : %d" % g_objCpTrade0721.GetHeaderValue(4 ))
    print("5 - (longlong)총결제예탁액          : %d" % g_objCpTrade0721.GetHeaderValue(5 ))
    print("6 - (longlong)현금위탁증거금        : %d" % g_objCpTrade0721.GetHeaderValue(6 ))
    print("7 - (longlong)대용위탁증거금        : %d" % g_objCpTrade0721.GetHeaderValue(7 ))
    print("8 - (longlong)총위탁증거금          : %d" % g_objCpTrade0721.GetHeaderValue(8 ))
    print("9 - (longlong)현금주문가능액        : %d" % g_objCpTrade0721.GetHeaderValue(9 ))
    print("10 - (longlong)대용주문가능액       : %d" % g_objCpTrade0721.GetHeaderValue(10))
    print("11 - (longlong)총주문가능액         : %d" % g_objCpTrade0721.GetHeaderValue(11))
    print("12 - (longlong)현금인출가능액       : %d" % g_objCpTrade0721.GetHeaderValue(12))
    print("13 - (longlong)대용인출가능액       : %d" % g_objCpTrade0721.GetHeaderValue(13))
    print("14 - (longlong)총인출가능액         : %d" % g_objCpTrade0721.GetHeaderValue(14))
    print("15 - (longlong)순위험증거금         : %d" % g_objCpTrade0721.GetHeaderValue(15))
    print("17 - (longlong)체결분순익           : %d" % g_objCpTrade0721.GetHeaderValue(17))
    print("18 - (longlong)미체결주문분         : %d" % g_objCpTrade0721.GetHeaderValue(18))
    print("20 - (longlong)현금미결제예탁액     : %d" % g_objCpTrade0721.GetHeaderValue(20))
    print("21 - (longlong)대용미결제예탁액     : %d" % g_objCpTrade0721.GetHeaderValue(21))
    print("22 - (longlong)총미결제예탁액       : %d" % g_objCpTrade0721.GetHeaderValue(22))
    print("23 - (longlong)현금결재순손실       : %d" % g_objCpTrade0721.GetHeaderValue(23))
    print("24 - (longlong)총결재순손실         : %d" % g_objCpTrade0721.GetHeaderValue(24))
    print("25 - (long) 수신개수                : %d" % g_objCpTrade0721.GetHeaderValue(25))
    print("26- (long) 매도신규주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(26))
    print("27- (long) 매도청산주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(27))
    print("28- (long) 매수신규주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(28))
    print("29- (long) 매수청산주문수량         : %d" % g_objCpTrade0721.GetHeaderValue(29))
    print("30- (longlong) 체결분순손실         : %d" % g_objCpTrade0721.GetHeaderValue(30))
    print("31 - (longlong) C등급총위험증거금   : %d" % g_objCpTrade0721.GetHeaderValue(31))
    print("32 - (longlong) C등급총위험가결산   : %d" % g_objCpTrade0721.GetHeaderValue(32))
    print("33 - (longlong) D등급총위험증거금   : %d" % g_objCpTrade0721.GetHeaderValue(33))
    print("34 - (longlong) D등급총위험가결산   : %d" % g_objCpTrade0721.GetHeaderValue(34))
    print("35 - (longlong) E등급총위험증거금   : %d" % g_objCpTrade0721.GetHeaderValue(35))
    print("36 - (longlong) E등급총위험가결산   : %d" % g_objCpTrade0721.GetHeaderValue(36))


    print("==========================================")
    print("계좌번호 : %s" % accNum)    
    g_objCpTrade0732.SetInputValue(0,accNum)
    g_objCpTrade0732.SetInputValue(1,"10")
    # 0 - (string)계좌번호
    # 1 - (string)상품관리구분[defaut:50]
    
    g_objCpTrade0732.BlockRequest()

    rqStatus = g_objCpTrade0732.GetDibStatus()
    rqRet = g_objCpTrade0732.GetDibMsg1()
    print("g_objCpTrade0732 통신상태", rqStatus, rqRet)
       

    print("- (string) 계좌번호           0  : %s" % g_objCpTrade0732.GetHeaderValue(0 ))
    print("- (string) 상품관리구분코드   1  : %s" % g_objCpTrade0732.GetHeaderValue(1 ))
    print("- (string) 계좌명             2  : %s" % g_objCpTrade0732.GetHeaderValue(2 ))
    print("- (longlong) 예수금           3  : %d" % g_objCpTrade0732.GetHeaderValue(3 ))
    print("- (long)미수금                4  : %d" % g_objCpTrade0732.GetHeaderValue(4 ))
    print("- (long)전일장내현금매도      5  : %d" % g_objCpTrade0732.GetHeaderValue(5 ))
    print("- (long)전일장내현금매수      6  : %d" % g_objCpTrade0732.GetHeaderValue(6 ))
    print("- (long)전일신용융자매도      7  : %d" % g_objCpTrade0732.GetHeaderValue(7 ))
    print("- (long)전일신용융자매수      8  : %d" % g_objCpTrade0732.GetHeaderValue(8 ))
    print("- (long)전일신용대주매도      9  : %d" % g_objCpTrade0732.GetHeaderValue(9 ))
    print("- (long)전일신용대주매수      10 : %d" % g_objCpTrade0732.GetHeaderValue(10))
    print("- (long)전일현금수수료        11 : %d" % g_objCpTrade0732.GetHeaderValue(11))
    print("- (long)전일현금제세금        12 : %d" % g_objCpTrade0732.GetHeaderValue(12))
    print("- (long)전일현금정산금        13 : %d" % g_objCpTrade0732.GetHeaderValue(13))
    print("- (long)전일장외단주매도      14 : %d" % g_objCpTrade0732.GetHeaderValue(14))
    print("- (long)전일장외단주매수      15 : %d" % g_objCpTrade0732.GetHeaderValue(15))
    print("- (long)전일장외수수료        16 : %d" % g_objCpTrade0732.GetHeaderValue(16))
    print("- (long)전일장외제세금        17 : %d" % g_objCpTrade0732.GetHeaderValue(17))
    print("- (long)전일장외정산금        18 : %d" % g_objCpTrade0732.GetHeaderValue(18))
    print("- (long)전일합계매도금        19 : %d" % g_objCpTrade0732.GetHeaderValue(19))
    print("- (long)전일합계매수금        20 : %d" % g_objCpTrade0732.GetHeaderValue(20))
    print("- (long)전일합계수수료        21 : %d" % g_objCpTrade0732.GetHeaderValue(21))
    print("- (long)전일합계제세금        22 : %d" % g_objCpTrade0732.GetHeaderValue(22))
    print("- (long)전일합계정산금        23 : %d" % g_objCpTrade0732.GetHeaderValue(23))
    print("- (long)전일장내현금신규융자  24 : %d" % g_objCpTrade0732.GetHeaderValue(24))
    print("- (long)전일신용융자융자상환  25 : %d" % g_objCpTrade0732.GetHeaderValue(25))
    print("- (long)전일장내현금신규대주  26 : %d" % g_objCpTrade0732.GetHeaderValue(26))
    print("- (long)전일신용융자대주상환  27 : %d" % g_objCpTrade0732.GetHeaderValue(27))
    print("- (long)전일장내현금신용상환  28 : %d" % g_objCpTrade0732.GetHeaderValue(28))
    print("- (long)전일상환융자이자      29 : %d" % g_objCpTrade0732.GetHeaderValue(29))
    print("- (long)전일상황이용료        30 : %d" % g_objCpTrade0732.GetHeaderValue(30))
    print("- (long)전일현금거래세        31 : %d" % g_objCpTrade0732.GetHeaderValue(31))
    print("- (long)전일대주소득세        32 : %d" % g_objCpTrade0732.GetHeaderValue(32))
    print("- (long)전일대주주민세        33 : %d" % g_objCpTrade0732.GetHeaderValue(33))
    print("- (long)금일장내현금매도      34 : %d" % g_objCpTrade0732.GetHeaderValue(34))
    print("- (long)금일장내현금매수      35 : %d" % g_objCpTrade0732.GetHeaderValue(35))
    print("- (long)금일신용융자매도      36 : %d" % g_objCpTrade0732.GetHeaderValue(36))
    print("- (long)금일신용융자매수      37 : %d" % g_objCpTrade0732.GetHeaderValue(37))
    print("- (long)금일신용대주매도      38 : %d" % g_objCpTrade0732.GetHeaderValue(38))
    print("- (long)금일신용대주매수      39 : %d" % g_objCpTrade0732.GetHeaderValue(39))
    print("- (long)금일현금수수료        40 : %d" % g_objCpTrade0732.GetHeaderValue(40))
    print("- (long)금일현금제세금        41 : %d" % g_objCpTrade0732.GetHeaderValue(41))
    print("- (long)금일현금정산금        42 : %d" % g_objCpTrade0732.GetHeaderValue(42))
    print("- (long)금일장외단주매도      43 : %d" % g_objCpTrade0732.GetHeaderValue(43))
    print("- (long)금일장외단주매수      44 : %d" % g_objCpTrade0732.GetHeaderValue(44))
    print("- (long)금일장외수수료        45 : %d" % g_objCpTrade0732.GetHeaderValue(45))
    print("- (long)금일장외제세금        46 : %d" % g_objCpTrade0732.GetHeaderValue(46))
    print("- (long)금일장외정산금        47 : %d" % g_objCpTrade0732.GetHeaderValue(47))
    print("- (long)금일합계매도금        48 : %d" % g_objCpTrade0732.GetHeaderValue(48))
    print("- (long)금일합계매수금        49 : %d" % g_objCpTrade0732.GetHeaderValue(49))
    print("- (long)금일합계수수료        50 : %d" % g_objCpTrade0732.GetHeaderValue(50))
    print("- (long)금일합계제세금        51 : %d" % g_objCpTrade0732.GetHeaderValue(51))
    print("- (long)금일합계정산금        52 : %d" % g_objCpTrade0732.GetHeaderValue(52))
    print("- (long)금일장내현금신규융자  53 : %d" % g_objCpTrade0732.GetHeaderValue(53))
    print("- (long)금일신용융자융자상환  54 : %d" % g_objCpTrade0732.GetHeaderValue(54))
    print("- (long)금일장내현금신규대주  55 : %d" % g_objCpTrade0732.GetHeaderValue(55))
    print("- (long)금일신용융자대주상환  56 : %d" % g_objCpTrade0732.GetHeaderValue(56))
    print("- (long)금일장내현금신용상환  57 : %d" % g_objCpTrade0732.GetHeaderValue(57))
    print("- (long)금일상환융자이자      58 : %d" % g_objCpTrade0732.GetHeaderValue(58))
    print("- (long)금일상황이용료        59 : %d" % g_objCpTrade0732.GetHeaderValue(59))
    print("- (long)금일현금거래세        60 : %d" % g_objCpTrade0732.GetHeaderValue(60))
    print("- (long)금일대주소득세        61 : %d" % g_objCpTrade0732.GetHeaderValue(61))
    print("- (long)금일대주주민세        62 : %d" % g_objCpTrade0732.GetHeaderValue(62))
    print("- (long) 익일영업일           63 : %d" % g_objCpTrade0732.GetHeaderValue(63))
    print("- (longlong) 익영업일예수금   64 : %d" % g_objCpTrade0732.GetHeaderValue(64))
    print("- (long) 결제일               65 : %d" % g_objCpTrade0732.GetHeaderValue(65))
    print("- (longlong) 결제일예수금     66 : %d" % g_objCpTrade0732.GetHeaderValue(66))

    print("==========================================")
    print("계좌번호 : %s" % accNum)    
    g_objCpTrade6722.SetInputValue(0,accNum)
    g_objCpTrade6722.SetInputValue(1,"106Q9") # 종목
    g_objCpTrade6722.SetInputValue(2,0) # 주문가격
    g_objCpTrade6722.SetInputValue(3,"2") # 시장가
    g_objCpTrade6722.SetInputValue(4,"50") # 상품종목
    g_objCpTrade6722.SetInputValue(5,"Y") # 수수료포함여부

    
    g_objCpTrade6722.BlockRequest()


    rqStatus = g_objCpTrade6722.GetDibStatus()
    rqRet = g_objCpTrade6722.GetDibMsg1()
    print("g_objCpTrade6722 통신상태", rqStatus, rqRet)
        

    print("%d" % g_objCpTrade6722.GetHeaderValue(2 ))
    print("%d" % g_objCpTrade6722.GetHeaderValue(3 ))
    print("%d" % g_objCpTrade6722.GetHeaderValue(4 ))
    print("%d" % g_objCpTrade6722.GetHeaderValue(11))
    print("%d" % g_objCpTrade6722.GetHeaderValue(12))
    print("%d" % g_objCpTrade6722.GetHeaderValue(13))
    print("%d" % g_objCpTrade6722.GetHeaderValue(14))
    print("%d" % g_objCpTrade6722.GetHeaderValue(15))
    print("%d" % g_objCpTrade6722.GetHeaderValue(16))
    print("%d" % g_objCpTrade6722.GetHeaderValue(17))
    print("%d" % g_objCpTrade6722.GetHeaderValue(18))
    print("%d" % g_objCpTrade6722.GetHeaderValue(19))
    print("%d" % g_objCpTrade6722.GetHeaderValue(20))
    print("%d" % g_objCpTrade6722.GetHeaderValue(21))
    print("%d" % g_objCpTrade6722.GetHeaderValue(22))
    print("%d" % g_objCpTrade6722.GetHeaderValue(23))
    print("%d" % g_objCpTrade6722.GetHeaderValue(24))
    print("%d" % g_objCpTrade6722.GetHeaderValue(25))
    print("%d" % g_objCpTrade6722.GetHeaderValue(26))
    print("%d" % g_objCpTrade6722.GetHeaderValue(27))
    print("%d" % g_objCpTrade6722.GetHeaderValue(28))
    print("%d" % g_objCpTrade6722.GetHeaderValue(29))
    print("%d" % g_objCpTrade6722.GetHeaderValue(30))


    


    print("예수금                            :   1000")    
    print("투자증거금 (예수금 * 매수강도)    :    700")
    print("계약당 증거금                     :    200")
    print("투자증거금 / 계약당 증거금        :   3.50")
    print("==========================================")
    print("매수 계약수                       :      3")
    print("최대 계약수                       :     50")
    print("max(매수,최대)                    :      3")
    print("주문가능 수량                     :      5")
    print("당기선물 잔존일이 0이면 매수하지 않는다.  ")
    print("")   
    print("")   
    
    


정상: 관리자권한으로 실행된 프로세스입니다.
계좌번호 : 777039828
g_objCpTrade0721 통신상태 0 3700 모의투자 조회완료 되었습니다(acc.bpm.blnbdwn.selfo)
3 - (longlong)현금결제예탁액        : 0
4 - (longlong)대용결제예탁액        : 0
5 - (longlong)총결제예탁액          : 0
6 - (longlong)현금위탁증거금        : 0
7 - (longlong)대용위탁증거금        : 0
8 - (longlong)총위탁증거금          : 0
9 - (longlong)현금주문가능액        : 0
10 - (longlong)대용주문가능액       : 0
11 - (longlong)총주문가능액         : 0
12 - (longlong)현금인출가능액       : 0
13 - (longlong)대용인출가능액       : 0
14 - (longlong)총인출가능액         : 0
15 - (longlong)순위험증거금         : 0
17 - (longlong)체결분순익           : 0
18 - (longlong)미체결주문분         : 0
20 - (longlong)현금미결제예탁액     : 0
21 - (longlong)대용미결제예탁액     : 0
22 - (longlong)총미결제예탁액       : 0
23 - (longlong)현금결재순손실       : 0
24 - (longlong)총결재순손실         : 0
25 - (long) 수신개수                : 0
26- (long) 매도신규주문수량         : 0
27- (long) 매도청산주문수량         : 0
28- (long) 매수신규주문수량         : 0
29- (long) 매수청산주문수량         : 0
30- (longlong) 체결분순손실         : 0
31 - (longlong) C등급총위험증거금